<a href="https://colab.research.google.com/github/Arvind2311/Machine_Learning/blob/master/tfServingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r /yolo
!ls /
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/",one_hot=True)

rm: cannot remove '/yolo': No such file or directory
1     content  etc   lib32  mnt   root	srv    tmp    var
bin   datalab  home  lib64  opt   run	swift  tools
boot  dev      lib   media  proc  sbin	sys    usr
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
from tensorflow.contrib import rnn
import tensorflow as tf

In [0]:
#Params
time_steps = 28
n_hidden = 128
n_input = 28
n_output = 10

#Network Params
learning_rate = 0.001
batch_size = 128

In [4]:
#Placeholders and variables
weights = tf.Variable(tf.random_normal([n_hidden, n_output]))
bias = tf.Variable(tf.random_normal([n_output]))

x = tf.placeholder(tf.float32, [None, time_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_output])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Making the input from [batch_size, time_steps, n_input] -> time_steps*[batch_size, n_input]
input = tf.unstack(x, time_steps, 1)

In [6]:
# Defining network
lstm_layer=rnn.BasicLSTMCell(n_hidden,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#converting last output of dimension [batch_size,n_hidden] to [batch_size,n_output] by out_weight multiplication
prediction=tf.matmul(outputs[-1],weights)+bias

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


In [0]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [8]:
#initialize variables
init=tf.global_variables_initializer()
!rm -r /2
builder = tf.saved_model.Builder("/yolo/1")
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<800:
        batch_x,batch_y=mnist.train.next_batch(batch_size=batch_size)

        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
    
    builder.add_meta_graph_and_variables(sess,
                                         ["serve"]
                                        )
    
    builder.save()
    sess.close()

rm: cannot remove '/2': No such file or directory
For iter  10
Accuracy  0.34375
Loss  1.8381665
__________________
For iter  20
Accuracy  0.4765625
Loss  1.4784718
__________________
For iter  30
Accuracy  0.546875
Loss  1.3266886
__________________
For iter  40
Accuracy  0.640625
Loss  1.0955858
__________________
For iter  50
Accuracy  0.6171875
Loss  1.1558287
__________________
For iter  60
Accuracy  0.765625
Loss  0.65772283
__________________
For iter  70
Accuracy  0.734375
Loss  0.6696626
__________________
For iter  80
Accuracy  0.8828125
Loss  0.47566122
__________________
For iter  90
Accuracy  0.8671875
Loss  0.5304836
__________________
For iter  100
Accuracy  0.8359375
Loss  0.45741713
__________________
For iter  110
Accuracy  0.8671875
Loss  0.4322006
__________________
For iter  120
Accuracy  0.84375
Loss  0.3906594
__________________
For iter  130
Accuracy  0.84375
Loss  0.4154219
__________________
For iter  140
Accuracy  0.8515625
Loss  0.45834693
__________________

In [9]:
!saved_model_cli show --dir /2 --tag_set serve
!ls

Traceback (most recent call last):
  File "/usr/local/bin/saved_model_cli", line 10, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 910, in main
    args.func(args)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 620, in show
    _show_signature_def_map_keys(args.dir, args.tag_set)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 77, in _show_signature_def_map_keys
    signature_def_map = get_signature_def_map(saved_model_dir, tag_set)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 243, in get_signature_def_map
    meta_graph = saved_model_utils.get_meta_graph_def(saved_model_dir, tag_set)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/tools/saved_model_utils.py", line 42, in get_meta_graph_def
    saved_model = reader.read_saved_model(saved_

In [10]:
with tf.Session() as sess1:
  tf.saved_model.loader.load(
      sess1,
      ["serve"],
      export_dir="/yolo/1"
  )
  
  #calculating test accuracy
  test_data = mnist.test.images[:128].reshape((-1, time_steps, n_input))
  test_label = mnist.test.labels[:128]
  print("Testing Accuracy:", sess1.run(accuracy, feed_dict={x: test_data, y: test_label}))

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /yolo/1/variables/variables
Testing Accuracy: 0.9765625


In [11]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2343  100  2343    0     0   6436      0 --:--:-- --:--:-- --:--:--  6436
OK
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:2 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.u

In [12]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tensorflow-model-server is already the newest version (1.13.0).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [13]:
!ls /
!tensorflow_model_server --rest_api_port=8501 --model_base_path=/yolo --model_name=saved_model

1     content  etc   lib32  mnt   root	srv    tmp    var
bin   datalab  home  lib64  opt   run	swift  tools  yolo
boot  dev      lib   media  proc  sbin	sys    usr
2019-04-14 05:28:13.053293: I tensorflow_serving/model_servers/server.cc:82] Building single TensorFlow model file config:  model_name: saved_model model_base_path: /yolo
2019-04-14 05:28:13.053535: I tensorflow_serving/model_servers/server_core.cc:461] Adding/updating models.
2019-04-14 05:28:13.053571: I tensorflow_serving/model_servers/server_core.cc:558]  (Re-)adding model: saved_model
2019-04-14 05:28:13.153848: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: saved_model version: 1}
2019-04-14 05:28:13.153884: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: saved_model version: 1}
2019-04-14 05:28:13.153908: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: saved_model version: 1}
2019-04-14 0